# Régression logistique sur "offers_with_10k_each_contracts"

Sans exploiter la description ni le titre des offres pour l'instant.

**PREPARATION DES DONNES**

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
#Import du fichier
AA = pd.read_csv("./../data/offers_with_10k_each_contracts.csv", encoding='utf-8')

In [3]:
#Répartition des contrats
AA.contract_id.value_counts()

3    10000
2    10000
1    10000
4     4800
5     3694
Name: contract_id, dtype: int64

In [4]:
#Nombre de valeurs "NaN" dans industry_id
AA.industry_id.isnull().value_counts()

False    38471
True        23
Name: industry_id, dtype: int64

In [5]:
#Nombre de valeurs "Nan" dans department_id
AA.department_id.isnull().value_counts()

False    31270
True      7224
Name: department_id, dtype: int64

In [6]:
#Nombre de valeurs "Nan" dans recruiter_id
AA.recruiter_id.isnull().value_counts()

True     28336
False    10158
Name: recruiter_id, dtype: int64

**PROBLEME :**

1/4 des valeurs pour department_id sont des valeurs "NaN". Il faut trouver une solution afin de ne pas perdre d'informations et de données. Cela semble moins dérangeant pour recruiter_id. A priori on ne prédit pas le contrat en fonction de l'entreprise qui recrute (quoi que ?). 

Une solution serait de remplacer les valeurs manquantes dans chaque colonne par les valeurs les plus fréquentes. Cela n'empêche pas que pour department_id, 1/4 des valeurs restent "faussées".

Donc : Est-ce vraiment pertinent ? Maybe, maybe not.

In [7]:
from sklearn.impute import SimpleImputer

In [13]:
AA = AA.sample(frac=1).reset_index(drop=True)
AA = AA.head(500)                    #Selection des 500 premières lignes mélangées au préalable sinon trop long à exécuter

#Ne fonctionne pas
#imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#imp.fit_transform(AA)

AA['target'] = AA['contract_id']     #Création d'une nouvelle colonne target qui nous sera utile par la suite

In [9]:
AA

,offer_id,title,description,contract_id,recruiter_id,department_id,industry_id,location,latitude,longitude,created_at,target
0,8759,H/F Technico Commercial,H/F TECHNICO COMMERCIAL / sud IDF lieu :\nvoi...,5,NaN,112.0,11.0,"AUBE, France",-8.811036,115.103803,2014-07-08 09:00:28.965776,5
1,562866,STAGE EN AGENCE BANCAIRE H/F,"Rejoignez le Crédit Agricole d'Ile-de-France, ...",3,NaN,NaN,5.0,Paris (75),48.851650,2.346154,2016-04-06 12:17:16.625674,3
2,135232,REDACTEUR FORMATIONS TECHNIQUES (H/F),"Entreprise :\n\n\nExperts, réseau spécialiste ...",2,NaN,104.0,9.0,"Carros, France",43.766800,7.191760,2014-09-21 16:59:17.651787,2
3,144043,Controleur de Gestion Industriel H/F,"Entreprise :\n\n\nAscot Consulting, cabinet de...",2,NaN,15.0,15.0,"Autour de Saint Lô, France",48.954881,-0.932603,2014-09-21 22:52:55.468181,2
4,8569,CONSEILLER(E) PATRIMONIAL H/F,Entreprise :\n\nGroupe bancaire d'envergure.\n...,4,NaN,112.0,14.0,"Aix-en-provence (1, France",-8.811036,115.103803,2014-07-08 08:59:16.484869,4
5,143440,H/F RESPONSABLE SERVICE JURIDIQUE,"Entreprise :\n\nEn rejoignant le CER FRANCE , ...",1,NaN,118.0,23.0,"CAHORS, France",44.447399,1.441840,2014-09-21 22:33:12.083994,1
6,618935,Chef de projet Communication (H/F),Au sein de la direction marketing et expérienc...,2,18551.0,103.0,24.0,"Meudon-la-Forêt, Meudon, France",48.787690,2.227747,2017-04-24 08:55:55.086457,2
7,699666,SAP Service Delivery Manager H/F,"En tant que SAP Service Delivery Manager, vous...",1,22863.0,111.0,9.0,"Lille, FR",50.629250,3.057256,2018-09-17 01:44:03.409701,1
8,312862,CONTROLEUR DE GESTION INTERNE H/F,Entreprise :\n\n\nLa CASQY (Communauté d'Agglo...,2,NaN,15.0,22.0,"TRAPPES (78190), France",48.776199,1.987150,2015-03-25 14:18:45.336361,2
9,692692,Développeur logiciel C++ F/H (H/F),Vous participez à la conception des logiciels ...,1,23069.0,110.0,10.0,"Rouen, France",49.447480,1.140530,2018-08-11 20:03:44.319636,1


In [11]:
#Remplacement des missing values par la valeur la plus fréquente pour chaque colonne
AA = AA.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [12]:
AA

,offer_id,title,description,contract_id,recruiter_id,department_id,industry_id,location,latitude,longitude,created_at,target
0,8759,H/F Technico Commercial,H/F TECHNICO COMMERCIAL / sud IDF lieu :\nvoi...,5,23069.0,112.0,11.0,"AUBE, France",-8.811036,115.103803,2014-07-08 09:00:28.965776,5
1,562866,STAGE EN AGENCE BANCAIRE H/F,"Rejoignez le Crédit Agricole d'Ile-de-France, ...",3,23069.0,1.0,5.0,Paris (75),48.851650,2.346154,2016-04-06 12:17:16.625674,3
2,135232,REDACTEUR FORMATIONS TECHNIQUES (H/F),"Entreprise :\n\n\nExperts, réseau spécialiste ...",2,23069.0,104.0,9.0,"Carros, France",43.766800,7.191760,2014-09-21 16:59:17.651787,2
3,144043,Controleur de Gestion Industriel H/F,"Entreprise :\n\n\nAscot Consulting, cabinet de...",2,23069.0,15.0,15.0,"Autour de Saint Lô, France",48.954881,-0.932603,2014-09-21 22:52:55.468181,2
4,8569,CONSEILLER(E) PATRIMONIAL H/F,Entreprise :\n\nGroupe bancaire d'envergure.\n...,4,23069.0,112.0,14.0,"Aix-en-provence (1, France",-8.811036,115.103803,2014-07-08 08:59:16.484869,4
5,143440,H/F RESPONSABLE SERVICE JURIDIQUE,"Entreprise :\n\nEn rejoignant le CER FRANCE , ...",1,23069.0,118.0,23.0,"CAHORS, France",44.447399,1.441840,2014-09-21 22:33:12.083994,1
6,618935,Chef de projet Communication (H/F),Au sein de la direction marketing et expérienc...,2,18551.0,103.0,24.0,"Meudon-la-Forêt, Meudon, France",48.787690,2.227747,2017-04-24 08:55:55.086457,2
7,699666,SAP Service Delivery Manager H/F,"En tant que SAP Service Delivery Manager, vous...",1,22863.0,111.0,9.0,"Lille, FR",50.629250,3.057256,2018-09-17 01:44:03.409701,1
8,312862,CONTROLEUR DE GESTION INTERNE H/F,Entreprise :\n\n\nLa CASQY (Communauté d'Agglo...,2,23069.0,15.0,22.0,"TRAPPES (78190), France",48.776199,1.987150,2015-03-25 14:18:45.336361,2
9,692692,Développeur logiciel C++ F/H (H/F),Vous participez à la conception des logiciels ...,1,23069.0,110.0,10.0,"Rouen, France",49.447480,1.140530,2018-08-11 20:03:44.319636,1


**REGRESSION LOGISTIQUE**

A TESTER

In [ ]:
#feature = ["department_id" , "industry_id"]

#features = AA[feature] 
#target = AA['target']

In [ ]:
#from sklearn.cross_validation import ShuffleSplit, train_test_split

#features_train, features_test, target_train, target_test = train_test_split(features, target)

In [ ]:
#Régression logistique

#cls = LogisticRegression() 
#cls.fit(features_train, target_train)
#predictions = cls.predict(features_test)
#cls.intercept_
#cls.coef_
#print(confusion_matrix(target_test, predictions))